Code to Calculate Mie Theory for Core-Shell spheres using theory from "Improved recursive algorithm for light scattering by a multilayered sphere" from Applied Optics Yang 2003

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
import os
import matplotlib.pyplot as plt
from pylab import *
import scipy.constants as sc
import scipy as scipy
from scipy import optimize
import matplotlib
from matplotlib.ticker import AutoMinorLocator
from matplotlib import gridspec
import matplotlib.ticker as ticker
%matplotlib inline

In [16]:
import cmath
import pandas as pd
import scipy.constants as sc
import scipy.special
import scipy
import math

inputdata = "/home/bldrdge1/inner_sphere_average_EXTABS_sphere.txt" #File containing refractive indices of material

data1 = pd.read_csv(inputdata, sep='\t', header=None, skiprows=1).values 
lamda = data1[:,0] #wavelengths in nm

column1 = int(input("Column contaning real refrac index of outermost shell:\t"))
column2 = int(input("Column contaning imaginary refrac index of outermost shell:\t"))
column3 = int(input("Column contaning real refrac index of middle shell:\t"))
column4 = int(input("Column contaning imaginary refrac index of middle shell:\t"))
column5 = int(input("Column contaning real refrac index of inner core:\t"))
column6 = int(input("Column contaning imaginary refrac index of inner core:\t"))

n_3 = data1[:,column1] #real part of the refractive index of outermost shell
k_3 = data1[:,column2] #imaginary part of the refractive index of outermost shell
N_3 = n_3+(k_3*1j) #complex refractive index of outermost shell

n_2 = data1[:,column3] #real part of the refractive index of middle shell
k_2 = data1[:,column4] #imaginary part of the refractive index of middle shell
N_2 = n_2+(k_2*1j) #complex refractive index of middle shell

n_1 = data1[:,column5] #real part of the refractive index of inner core
k_1 = data1[:,column6] #imaginary part of the refractive index of inner core
N_1 = n_1+(k_1*1j) #complex refractive index of inner sphere

n_m = 1.33 #real part of the refractive index of medium
k_m = 0 #imaginary part of refractive index of medium
N_m = n_m+(k_m*1j)

m_m = 1 #relative refractive index of medium
m_3 = N_3/N_m
m_2 = N_2/N_m
m_1 = N_1/N_m

linemax = 702

output = "\[Lambda] QEXT QNF-OUT QNF-INT QNF-IN QSCA QABS CEXT CSCA CABS"

r_3 = float(input("Outer radius of the outermost shell:\t")) #outer radius of outer shell
r_2 = float(input("Outer radius of the middle shell:\t")) #outer radius of middle shell
r_1 = float(input("Radius of the inner core sphere:\t")) #radius of inner sphere

def x_1(r): return (2*sc.pi*r*N_m)/lamda
def x_2(r): return (2*sc.pi*r*N_m)/lamda
def x_3(r): return (2*sc.pi*r*N_m)/lamda
def y_1(r): return (m_1*2*sc.pi*r*N_m)/lamda
def y_2(r): return (m_2*2*sc.pi*r*N_m)/lamda
def y_3(r): return (m_3*2*sc.pi*r*N_m)/lamda
def z_1(r): return (m_2*2*sc.pi*r*N_m)/lamda
def z_2(r): return (m_3*2*sc.pi*r*N_m)/lamda
def z_3(r): return (m_m*2*sc.pi*r*N_m)/lamda

largest = max(max(x_1(r_1).real), 
              max(x_2(r_2).real), 
              max(x_3(r_3).real), 
              max(y_1(r_1).real), 
              max(y_2(r_2).real), 
              max(y_3(r_3).real), 
              max(z_1(r_1).real), 
              max(z_2(r_2).real), 
              max(z_3(r_3).real))
lastTerm = math.ceil(abs(largest+4.05*(largest**(1/3))+2))
#The previous two variables were chosen from other Mie-code

# def phi(n, z): return z*scipy.special.spherical_jn(n, z) #Riccati-Bessel Function where n is the order and z is the parameter
# def Dphi(n, z): return scipy.special.spherical_jn(n, z, True) #Derivative of Bessel Fxn
# def chi(n, z): return -z*scipy.special.spherical_jn(n, z) #Negative Bessel Fxn
# def Dchi(n, z): return -scipy.special.spherical_jn(n, z) #Derivative of negative Bessel Fxn
# def zeta(n, z): return z*scipy.special.spherical_jn(n,z)+1j*scipy.special.spherical_yn(n,z)
# def Dzeta(n, z): return derivative
# def D_1(n, z): return Dphi(n, z)/phi(n, z)
# def D_3(n, z): return Dzeta(n, z)/zeta(n, z)
# def RR(n, z): return phi(n, z)/zeta(n, z)

xTable = np.vstack([np.round([phi(n, x_1(r_1)), Dphi(n, x_1(r_1)), 
                    zeta(n, x_1(r_1)), Dzeta(n, x_1(r_1)), 
                    phi(n, x_2(r_2)), Dphi(n, x_2(r_2)), 
                    zeta(n, x_2(r_2)), Dzeta(n, x_2(r_2))]) for n in np.arange(1,5)])

yTable = np.vstack([np.round([phi(n, y_1(r_1)), Dphi(n, y_1(r_1)), 
                    zeta(n, y_1(r_1)), Dzeta(n, y_1(r_1)), 
                    phi(n, y_2(r_2)), Dphi(n, y_2(r_2)), 
                    zeta(n, y_2(r_2)), Dzeta(n, y_2(r_2))]) for n in np.arange(1,5)])

zTable = np.vstack([np.round([phi(n, z_1(r_1)), Dphi(n, z_1(r_1)), 
                    zeta(n, z_1(r_1)), Dzeta(n, z_1(r_1)), 
                    phi(n, z_2(r_2)), Dphi(n, z_2(r_2)), 
                    zeta(n, z_2(r_2)), Dzeta(n, z_2(r_2))]) for n in np.arange(1,5)])

d1Table = np.vstack([np.round([D_1(n, y_1(r_1)), D_1(n, y_2(r_2)), 
                    D_1(n, z_1(r_1)), D_1(n, z_2(r_2))]) for n in np.arange(1,5)])

d3Table = np.vstack([np.round([D_3(n, y_1(r_1)), D_3(n, y_2(r_2)), 
                    D_3(n, z_1(r_1)), D_3(n, z_2(r_2))]) for n in np.arange(1,5)])

rrTable = np.vstack([np.round([RR(n, y_1(r_1)), RR(n, y_2(r_2)), 
                    RR(n, z_1(r_1)), RR(n, z_2(r_2))]) for n in np.arange(1,5)])

def A_2(n, r): return (RR(n, z_1(r)))*((m_2*D_1(n, y_1(r))-m_1*D_1(n, z_1(r)))/(m_2*D_1(n, y_1(r))-m_1*D_3(n, z_1(r))))


Column contaning real refrac index of outermost shell:	1
Column contaning imaginary refrac index of outermost shell:	1
Column contaning real refrac index of middle shell:	1
Column contaning imaginary refrac index of middle shell:	1
Column contaning real refrac index of inner core:	1
Column contaning imaginary refrac index of inner core:	1
Outer radius of the outermost shell:	1
Outer radius of the middle shell:	1
Radius of the inner core sphere:	1
[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
